In [1]:
import os

In [2]:
%pwd

'/home/jonnyoh/code/JonnyPOH/End-to-end-Machine-Learning-Project-with-MLflow/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/jonnyoh/code/JonnyPOH/End-to-end-Machine-Learning-Project-with-MLflow'

In [5]:
import mlflow
# custom token = f4b855696c23d4f486278a77b4d1afedcc2a2cba
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/JonnyPOH/End-to-end-Machine-Learning-Project-with-MLflow.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'JonnyPOH'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'f4b855696c23d4f486278a77b4d1afedcc2a2cba'
mlflow.set_tracking_uri(os.environ['MLFLOW_TRACKING_URI'])

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH):

        print(f"Loading configuration from: {config_filepath}")
        print(f"Loading parameters from: {params_filepath}")
        print(f"Loading schema from: {schema_filepath}")

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/JonnyPOH/End-to-end-Machine-Learning-Project-with-MLflow",
        )

        return model_evaluation_config


In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config


    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2



    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():
            print("MLflow run started.")
            predicted_qualities = model.predict(test_x)
            print("Model prediction completed.")

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            print(f"Evaluation metrics calculated: RMSE={rmse}, MAE={mae}, R2={r2}")

            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)
            print("Json saved.")

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)
            print("Metrics logged to MLflow.")

            print(f"Tracking URL type store: {tracking_url_type_store}")

            # Model registry logic
            try:
                if tracking_url_type_store != "file":
                    # Register the model if not using file store
                    mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
                    print("Model logged and registered in MLflow.")
                else:
                    mlflow.sklearn.log_model(model, "model")
                    print("Model logged in MLflow.")
            except Exception as e:
                print("An error occurred while logging the model:")
                print(e)

        print("MLflow run completed.")


In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

Loading configuration from: config/config.yaml
Loading parameters from: params.yaml
Loading schema from: schema.yaml
[2024-07-10 12:05:27,437: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-10 12:05:27,444: INFO: common: yaml file: params.yaml loaded successfully]


[2024-07-10 12:05:27,450: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-10 12:05:27,453: INFO: common: created directory at: artifacts]
[2024-07-10 12:05:27,455: INFO: common: created directory at: artifacts/model_evaluation]


/home/jonnyoh/code/JonnyPOH/End-to-end-Machine-Learning-Project-with-MLflow/venvend/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator ElasticNet from version 1.4.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


MLflow run started.
Model prediction completed.
Evaluation metrics calculated: RMSE=0.7444251162404556, MAE=0.579432761841836, R2=0.24096869786383246
[2024-07-10 12:05:28,164: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]
Json saved.
Metrics logged to MLflow.
Tracking URL type store: https
An error occurred while logging the model:
API request to endpoint /api/2.0/mlflow/registered-models/create failed with error code 404 != 200. Response body: '<!DOCTYPE html>
<html lang="en-US">
  <head data-suburl="">
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8"/>
    <meta http-equiv="X-UA-Compatible" content="IE=edge"/>
    
    
      <meta name="viewport" content="width=device-width, initial-scale=1"/>
    
    <meta name="referrer" content="no-referrer"/>
    <meta name="_csrf" content="NNAPCZvXFdRHdwwz4Wlhc99FYz06MTcyMDYwOTUyNzg2NjY3MjgwMw"/>
    <meta name="_suburl" content=""/>
    <meta name="_appversion" content="3bbbb1ade33045c30e81b4